In [1]:
#imports
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection, metrics
from imblearn.over_sampling import SMOTE
import csv

In [4]:
#import data

positives_dna = []
negatives_dna = []
positives_enzyme = []
negatives_enzyme = []

with open('data/minimal_model_pdidb_positives.txt') as p_dna:
    for row in csv.reader(p_dna,delimiter="\t"):
        positives_dna.append(row)

with open('data/minimal_model_pdidb_negatives.txt') as n_dna:
    for row in csv.reader(n_dna,delimiter="\t"):
       negatives_dna.append(row)
       
with open('data/minimal_model_enzyme_positives.txt') as p_enzyme:
    for row in csv.reader(p_enzyme,delimiter="\t"):
        positives_enzyme.append(row)
        
with open('data/minimal_model_enzyme_negatives.txt') as n_enzyme:
    for row in csv.reader(n_enzyme,delimiter="\t"):
        negatives_enzyme.append(row)

In [5]:
#preprocess data

tmp_dna = positives_dna + negatives_dna
tmp_enzyme = positives_enzyme + negatives_enzyme

labels_dna = [1]*len(positives_dna)+[0]*len(negatives_dna)
labels_enzyme = [1]*len(positives_enzyme)+[0]*len(negatives_enzyme)

dna = []
enzyme = []

for j in tmp_dna:
    new_j = [float(i) for i in j]
    dna.append(new_j)
    
for j in tmp_enzyme:
    new_j = [float(i) for i in j]
    enzyme.append(new_j)

In [6]:
#train and test datasets

dna_x_train,dna_y_train,dna_x_test,dna_y_test = model_selection.train_test_split(dna, labels_dna, test_size=0.2, random_state=6)
enzyme_x_train,enzyme_y_train,enzyme_x_test,enzyme_y_test = model_selection.train_test_split(enzyme,labels_enzyme,test_size=0.2,random_state=6)

x_train = dna_x_train + enzyme_x_train
y_train = dna_y_train + enzyme_y_train
x_test = dna_x_test + enzyme_x_test
y_test = dna_y_test + enzyme_y_test

sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_sample(x_train,y_train)

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=3)
parameters = {'alpha': [0.0001], 'random_state': [1]}

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').